In [8]:
import numpy as np
import pandas as pd
import json

# dataset coronavirus WHO
pd.set_option('max_colwidth', 100)  # Increase column width
with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FileNotFoundError: [Errno 2] No such file or directory: 'intents.json'